In [1]:
from getSeeds import getUserSeeds
from GraphCut import iterativeGraphcut
import cv2
import numpy as np
from closed_form_matting import closed_form_matting_with_trimap

In [331]:
def save_results(image_path,image_orig,masks,denoised_masks,suffix):
    #image_orig=cv2.imread(image_path)
    orig_size=image_orig.shape[0:2]
    for i in range(len(masks)):
        im=np.where(cv2.resize(masks[i],tuple(reversed(orig_size)))[...,None]==1,image_orig,1)
        im1=np.where(cv2.resize(denoised_masks[i],tuple(reversed(orig_size)))[...,None]==1,image_orig,1)
        cv2.imwrite(image_path[:-4]+f'_mask_segmented_{i}'+suffix+'.jpg',im)
        cv2.imwrite(image_path[:-4]+f'_denoisedMask_segmented_{i}'+suffix+'.jpg',im1)

def run(num_iter,tol,image_path,image_size,pointer_radius,num_neighbors,weights,denoise,gamma,hsv=False,suffix=''):
    seed,_,r,image_orig,image_resized=getUserSeeds(image_path,image_size,pointer_radius=pointer_radius,hsv=hsv)
    flows,masks,denoised_masks,FNs=iterativeGraphcut(num_iter,tol,seed,image_resized,num_neighbors,gamma,r,weights,NX=True,denoise=denoise,hsv=hsv)
    save_results(image_path,image_orig,masks,denoised_masks,suffix)
    return flows, masks,denoised_masks,FNs

def runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights,denoise,gamma,hsv=False,suffix=''):
    flows,masks,denoised_masks,FNs=iterativeGraphcut(num_iter,tol,seed,image_resized,num_neighbors,gamma,r,weights,NX=True,denoise=denoise,hsv=hsv)
    save_results(image_path,image_orig,masks,denoised_masks,suffix)
    return flows, masks,denoised_masks,FNs

def generate_trimap(_mask,eroision_iter=6,dilate_iter=8,k=3,pre_erode_k=4,pre_erode_iter=1,pre_dilate_k=3, pre_dilate_iter=1,dilate_before=False):
    mask=cv2.copyTo(_mask,None,None)
    mask[mask==1] = 255
    d_kernel = np.ones((k,k))
    if not  dilate_before:
        mask=cv2.erode(mask,np.ones((pre_erode_k,pre_erode_k)),iterations=pre_erode_iter)
        mask=cv2.dilate(mask,np.ones((pre_dilate_k,pre_dilate_k)),iterations=pre_dilate_iter)
    else:
        mask=cv2.dilate(mask,np.ones((pre_dilate_k,pre_dilate_k)),iterations=pre_dilate_iter)
        mask=cv2.erode(mask,np.ones((pre_erode_k,pre_erode_k)),iterations=pre_erode_iter)
    erode  = cv2.erode(mask,d_kernel,iterations=eroision_iter)
    dilate = cv2.dilate(mask,d_kernel,iterations=dilate_iter)
    unknown1 = cv2.bitwise_xor(erode,mask)
    unknown2 = cv2.bitwise_xor(dilate,mask)
    unknowns = cv2.add(unknown1,unknown2)
    unknowns[unknowns==255]=127
    trimap = cv2.add(mask,unknowns)
    # cv2.imwrite("mask.png",mask)
    # cv2.imwrite("dilate.png",dilate)
    # cv2.imwrite("tri.png",trimap)
    labels = trimap.copy()
    labels[trimap==127]=1 #unknown
    labels[trimap==255]=2 #foreground
    #cv2.imwrite(mask_path,labels)
    return labels,trimap
def border_matting(image_path,mask,eroision_iter=2,dilate_iter=2,k=3,pre_erode_k=5,pre_erode_iter=1,pre_dilate_k=3, pre_dilate_iter=1,dilate_before=False):
    _,t=generate_trimap(mask,eroision_iter,dilate_iter,k,pre_erode_k,pre_erode_iter,pre_dilate_k,pre_dilate_iter,dilate_before)
    #cv2.imshow('',t)
    #cv2.waitKey(0)
    image_orig = cv2.imread(image_path)
    t=cv2.resize(t,tuple(reversed(image_orig.shape[0:2])))
    print(t.shape,image_orig.shape)
    alpha = closed_form_matting_with_trimap(image_orig,t/255)
    new_seg = cv2.cvtColor(image_orig,cv2.COLOR_BGR2BGRA)
    new_seg[:,:,3]=alpha*255
    cv2.imwrite(f"{image_path[:-4]}_segmented_final.png", new_seg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


#### Weight Experiments

In [3]:
image_path='./images/banana.jpg'
image_size=(256,256)
radius=3
hsv=False
num_iter= 1
tol=10
gamma=100000
denoise=False
num_neighbors=1

In [ ]:
seed,_,r,image_orig,image_resized=getUserSeeds(image_path,image_size,pointer_radius=radius,hsv=hsv)

In [12]:
weights=(1.1,1.2)
weights2=(1.9,1.2)
weights3=(1,2.5)
res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights,denoise,gamma,hsv,str(weights))
res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights2,denoise,gamma,hsv,str(weights2))
res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights3,denoise,gamma,hsv,str(weights3))

beta= 0.016782060909975232
50.18773365020752 seconds to create graph
6.995261907577515 seconds to run graphCutV1
Flow delta: 747901.7000001009, Flow 747901.7000001009
59.50693988800049 seconds to run iterative graphCut
beta= 0.016782060909975232
55.94126296043396 seconds to create graph
7.292559623718262 seconds to run graphCutV1
Flow delta: 1093014.4999999902, Flow 1093014.4999999902
65.01964664459229 seconds to run iterative graphCut
beta= 0.016782060909975232
52.41720390319824 seconds to create graph
6.888342618942261 seconds to run graphCutV1
Flow delta: 866446.0, Flow 866446.0
61.82341265678406 seconds to run iterative graphCut


#### Neighbor Experiments

In [294]:
image_path='./images/llama.jpg'
image_size=(256,256)
radius=3
hsv=False
weights=[(2,1.2),(2,1.2)]
num_iter= 1
tol=10
denoise=False
gamma=[10,10]

In [ ]:
seed,_,r,image_orig,image_resized=getUserSeeds(image_path,image_size,pointer_radius=radius,hsv=hsv)

In [296]:
num_neighbors1=[1]
num_neighbors2=[3]
res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors1,weights,denoise,gamma,hsv,f'_neighbors_{8}')
#res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors2,weights,denoise,gamma,hsv,f'_neighbors_{48}')

beta= 0.002268503680398001
41.702632904052734 seconds to create graph
# of fg and bg pix 13953,8741
6.717160224914551 seconds to run graphCutV1
Flow delta: 354776.3999150572, Flow 354776.3999150572
52.26901817321777 seconds to run iterative graphCut


#### Gamma Experiments

In [310]:
image_path='./images/lena.jpg'
image_size=(256,256)
radius=8
hsv=False
weights=[(3,1),(1,1),(0.95,1.05),(1,1),(1,1)]#[(3,1),(1.2,1),(0.95,1.05),(0.95,1),(0.95,1)] #[(1,1),(1,1),(1,1),(1,1)]#
num_iter= 5
tol=1e-3
denoise=True
num_neighbors=[1,2,3,2,3] #[2,2,2,2]#

In [308]:
seed,_,r,image_orig,image_resized=getUserSeeds(image_path,image_size,pointer_radius=radius,hsv=hsv)

select ROI.
Seeds for FG
Seeds for BG


In [311]:
gamma=[1,50,100,100,100,50]
#gamma=[1,1,1]
gamma1=[50,50,50]
gamma2=[100,100,100]
gamma3=[500,500,500]
res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights,denoise,gamma,hsv,f'_gamma_{gamma[0]}')
# res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights,denoise,gamma1,hsv,f'_gamma_{gamma1[0]}')
# res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights,denoise,gamma2,hsv,f'_gamma_{gamma2[0]}')
# res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights,denoise,gamma3,hsv,f'_gamma_{gamma3[0]}')
#border_matting(image_path,res[2][4],2,2,3,5,1,3,2)

beta= 0.0022132013725211573
53.660269021987915 seconds to create graph
# of fg and bg pix 41023,3016
17.255286693572998 seconds to run graphCutV1
Flow delta: 500465.8205774786, Flow 500465.8205774786
100.61236238479614 seconds to create graph
# of fg and bg pix 37205,6834
11.645330667495728 seconds to run graphCutV1
Flow delta: 117247.25116386765, Flow 383218.56941361097
105.7181351184845 seconds to create graph
# of fg and bg pix 28571,15468
12.738853693008423 seconds to run graphCutV1
Flow delta: 10507.79779694439, Flow 393726.36721055536
111.48103761672974 seconds to create graph
# of fg and bg pix 31226,12813
16.169471263885498 seconds to run graphCutV1
Flow delta: 9746.19500992092, Flow 383980.17220063444
110.00148463249207 seconds to create graph
# of fg and bg pix 31218,12821
16.842244863510132 seconds to run graphCutV1
Flow delta: 2495.31432134466, Flow 381484.8578792898
573.0913133621216 seconds to run iterative graphCut


In [358]:
border_matting(image_path,res[2][4],2,3,5,15,2,2,13,dilate_before=True)

(256, 256) (256, 256, 3)


#### HSV vs RGB Experiments

In [49]:
image_path='./images/llama.jpg'
image_size=(256,256)
radius=3
weights=(2.5,1.2)
num_iter= 1
tol=10
denoise=False
num_neighbors=2
gamma=50

In [54]:
seed,_,r,image_orig,image_resized=getUserSeeds(image_path,image_size,pointer_radius=radius,hsv=False)

select ROI.
Seeds for FG
Seeds for BG


In [55]:
hsv=False
hsv1=True
res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights,denoise,gamma,hsv,f'_rgb')
image_resize= cv2.cvtColor(image_resized, cv2.COLOR_BGR2HSV)
res=runExperiment(num_iter,tol,seed,r,image_orig,image_resize,num_neighbors,weights,denoise,gamma,hsv1,f'_hsv')


beta= 0.0053384988155315005
53.64217805862427 seconds to create graph
# of fg and bg pix 10252,13461
16.757009744644165 seconds to run graphCutV1
Flow delta: 535303.9004690528, Flow 535303.9004690528
74.84968185424805 seconds to run iterative graphCut
beta= 0.005178098406410078
56.31271696090698 seconds to create graph
# of fg and bg pix 10170,13543
17.10338068008423 seconds to run graphCutV1
Flow delta: 592478.6433732454, Flow 592478.6433732454
78.01223587989807 seconds to run iterative graphCut


#### CORPUS CALLOSUM SEGMENTATION

In [245]:
image_path='./images/corpus_callosum_27.png'
# image_size=(256,256)
# radius=3
# weights=(1.1,1.2)
# num_iter= 5
# tol=10
# denoise=False
# num_neighbors=3
# gamma=1
# hsv=False
image_size=(256,256)
radius=3
hsv=False
weights=[(3,1),(1.2,1),(0.95,1.05),(0.95,1),(0.95,1)]
num_iter= 5
tol=1e-3
denoise=True
num_neighbors=[1,2,3,2,3]
gamma=[1,50,100,100,100,50]

In [246]:
seed,_,r,image_orig,image_resized=getUserSeeds(image_path,image_size,pointer_radius=radius,hsv=False)

select ROI.
Seeds for FG
Seeds for BG


In [247]:
res=runExperiment(num_iter,tol,seed,r,image_orig,image_resized,num_neighbors,weights,denoise,gamma,hsv)
border_matting(image_path,res[2][4],1,0,2,0,0,2,1)

beta= 0.0035107192926021923
37.56923699378967 seconds to create graph
# of fg and bg pix 28344,26858
25.192928552627563 seconds to run graphCutV1
Flow delta: 1513929.0, Flow 1513929.0
106.88254594802856 seconds to create graph
# of fg and bg pix 28627,26575
25.750194311141968 seconds to run graphCutV1
Flow delta: 1075269.3847013097, Flow 438659.61529869033
115.0191216468811 seconds to create graph
# of fg and bg pix 28513,26689
23.895692825317383 seconds to run graphCutV1
Flow delta: 20386.65557480679, Flow 418272.95972388354
114.2289969921112 seconds to create graph
# of fg and bg pix 28507,26695
26.52107000350952 seconds to run graphCutV1
Flow delta: 13839.02912613214, Flow 404433.9305977514
124.05455160140991 seconds to create graph
# of fg and bg pix 28501,26701
22.58618974685669 seconds to run graphCutV1
Flow delta: 18.72087391396053, Flow 404415.20972383744
640.2501883506775 seconds to run iterative graphCut
(175, 288) (175, 288, 3)


In [274]:
border_matting(image_path,res[2][4],0,1,2,2,1,0,0)

(175, 288) (175, 288, 3)


In [6]:

cv2.imshow('',np.where(seed[...,None]==2,image_resized,0))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
FN= res[3][0]
[(u,v,FN.edges[u,v]['capacity']) for u,v in FN.edges]

[('(-1, -1)', '(128, 87)', 7.700000000000001),
 ('(-1, -1)', '(128, 88)', 8.8),
 ('(-1, -1)', '(128, 89)', 9.9),
 ('(-1, -1)', '(128, 90)', 11.0),
 ('(-1, -1)', '(128, 91)', 11.0),
 ('(-1, -1)', '(128, 92)', 11.0),
 ('(-1, -1)', '(128, 93)', 11.0),
 ('(-1, -1)', '(128, 94)', 11.0),
 ('(-1, -1)', '(128, 95)', 9223372036854775807),
 ('(-1, -1)', '(128, 96)', 9.9),
 ('(-1, -1)', '(128, 97)', 8.8),
 ('(-1, -1)', '(128, 98)', 8.8),
 ('(-1, -1)', '(128, 99)', 8.8),
 ('(-1, -1)', '(128, 100)', 8.8),
 ('(-1, -1)', '(128, 101)', 8.8),
 ('(-1, -1)', '(128, 102)', 8.8),
 ('(-1, -1)', '(128, 103)', 8.8),
 ('(-1, -1)', '(128, 104)', 8.8),
 ('(-1, -1)', '(128, 105)', 7.700000000000001),
 ('(-1, -1)', '(128, 106)', 8.8),
 ('(-1, -1)', '(128, 107)', 8.8),
 ('(-1, -1)', '(128, 108)', 8.8),
 ('(-1, -1)', '(128, 109)', 8.8),
 ('(-1, -1)', '(128, 110)', 8.8),
 ('(-1, -1)', '(128, 111)', 8.8),
 ('(-1, -1)', '(128, 112)', 8.8),
 ('(-1, -1)', '(128, 113)', 8.8),
 ('(-1, -1)', '(128, 114)', 8.8),
 ('(-1, -1)'